In [10]:
import qsharp.azure
targets = qsharp.azure.connect(
   resourceId="/subscriptions/b0c3bbe3-7123-4ba4-9aa2-57d9845f4a1d/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/qchack-2022-b",
   location="eastus")

Preparing Q# environment...
.Authenticated using Microsoft.Azure.Quantum.Authentication.TokenFileCredential


Connected to Azure Quantum workspace qchack-2022-b in location eastus.


In [13]:
import qsharp

print("This workspace's targets:")
for target in targets:
    print("-", target.id)

This workspace's targets:
- ionq.qpu
- ionq.simulator
- quantinuum.hqs-lt-s1
- quantinuum.hqs-lt-s1-apival
- quantinuum.hqs-lt-s2
- quantinuum.hqs-lt-s2-apival
- quantinuum.hqs-lt-s1-sim
- quantinuum.hqs-lt-s2-sim
- quantinuum.hqs-lt


In [14]:
%%qsharp
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;

operation Task1 (x : Qubit[], y : Qubit) : Unit is Adj + Ctl {
    Controlled X([x[0]],y);
    Controlled X([x[1]],y);
    X(y);
}

In [18]:
%%qsharp
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;

operation VerifySingleOutputFunction(
    numInputs : Int, 
    op : (Qubit[], Qubit) => Unit is Adj+Ctl, 
    predicate : Int -> Bool
) : Unit {
    for assignment in 0 .. 2^numInputs - 1 {
        use (inputs, output) = (Qubit[numInputs], Qubit());
        within {
            ApplyXorInPlace(assignment, LittleEndian(inputs));
            AllowAtMostNCallsCA(0, Measure, "You are not allowed to use measurements");
        } apply {
            op(inputs, output);
        }

        // Check that the result is expected
        let actual = ResultAsBool(MResetZ(output));
        let expected = predicate(assignment);
        Fact(actual == expected,
            $"Oracle evaluation result {actual} does not match expected {expected} for assignment {IntAsBoolArray(assignment, numInputs)}");

        // Check that the inputs were not modified
        Fact(MeasureInteger(LittleEndian(inputs)) == 0, 
            $"The input states were modified for assignment {assignment}");
    }
}


// ------------------------------------------------------
function Is03ModFour (input : Int) : Bool {
    return input % 4 == 0 or input % 4 == 3;
}

// ------------------------------------------------------
operation TestIs03ModFour() : Unit {
    for i in 2 .. 5 {
        VerifySingleOutputFunction(i, Task1, Is03ModFour);
    }
    Message("Success!");
}



In [21]:
%%qsharp
%toffoli TestIs03ModFour

Success!
